In [1]:
import pandas
pandas.set_option("display.max_rows", 50, "display.max_columns", 50)

import numpy
import pygsheets

import Pipelines_Current dataset

In [26]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')
spreadsheet = gc.open_by_key('18od3L-Uy-N0K0mnUIaeBWLVmZKeaLnjG9rOVSOjSSy4')

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df()
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df()
#owners = spreadsheet[2].get_as_df()

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)

# delete columns that aren't the same in the sheets, to concatenate them...
columns_not_in_oil = list(set(gas_pipes.columns)-set(oil_pipes.columns))
columns_not_in_gas = list(set(oil_pipes.columns)-set(gas_pipes.columns))
gas_pipes.drop(columns=columns_not_in_oil, axis=1, inplace=True)
oil_pipes.drop(columns=columns_not_in_gas, axis=1, inplace=True)

pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Owner').get_as_df()
owners_df_orig.set_index('ProjectID', inplace=True)

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
owners_df_orig = owners_df_orig[owners_df_orig.index!='']

regions_df_orig = spreadsheet.worksheet('title', 'Region dictionary').get_as_df()

In [27]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig[pipes_df_orig['Route']!='N/A']
owners_df_orig = owners_df_orig[owners_df_orig.index!='']

### create dataframes for region, country_ratios_df, owners_df

In [28]:
country_ratios_df_touse = country_ratios_df.copy()#[country_ratios_df['Country'].str.contains(
                                            #'|'.join(region_df_touse['Country'].tolist()))]

owners_df_touse = owners_df_orig.copy()#[owners_df_orig['Countries'].str.contains(
                                       #     '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.copy()#[pipes_df_orig['Countries'].str.contains(
                                     #       '|'.join(region_df_touse['Country'].tolist()))]

### set up info to analyze Owners tab

In [29]:
# get a set list of all owners
owner_column_list = ['Owner1', 
                     'Owner2', 
                     'Owner3', 
                     'Owner4', 
                     'Owner5', 
                     'Owner6', 
                     'Owner7', 
                     'Owner8', 
                     'Owner9', 
                     'Owner10', 
                     'Owner11']

percent_column_list = ['Percent1', 
                       'Percent2', 
                       'Percent3', 
                       'Percent4', 
                       'Percent5', 
                       'Percent6', 
                       'Percent7', 
                       'Percent8', 
                       'Percent9', 
                       'Percent10', 
                       'Percent11']

owner_id_list = ['ID1', 
                 'ID2', 
                 'ID3', 
                 'ID4', 
                 'ID5', 
                 'ID6', 
                 'ID7', 
                 'ID8', 
                 'ID9', 
                 'ID10', 
                 'ID11']

### sum MergedKmByCountry and MergedKmByRegion

In [33]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['PipelineBubbleRegion'])))

In [34]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

In [35]:
regions_df_orig['Country']=='USA'

0      False
1      False
2      False
3      False
4      False
       ...  
264    False
265    False
266    False
267    False
268    False
Name: Country, Length: 269, dtype: bool

In [38]:
fuel_type = 'Gas'
fuel_type = 'Oil'
#fuel_type = 'NGL'

country_ratios_df_subset = country_ratios_df_touse.copy()[country_ratios_df_touse['Fuel']==fuel_type]
#for idx,row in country_ratios_df_subset.iterrows():
#    country_ratios_df_subset['Region'] = regions_df_orig.loc[regions_df_orig['Country']==row['Country'],'Region']

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[
        country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby(
        'Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby('PipelineBubbleRegion')['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.name = 'Region'
km_by_country.index.name = 'Country'

totals_row = km_by_region.sum(axis=0)
totals_row.name = 'Total'
km_by_region = km_by_region.append(totals_row)

totals_row = km_by_country.sum(axis=0)
totals_row.name = 'Total'
km_by_country = km_by_country.append(totals_row)

km_by_region.to_excel(fuel_type+'-PipelineKmByRegionAndStatus-OldRegionsForScott.xlsx')
#km_by_country.to_excel('GGIT'+'-PipelineKmByCountryAndStatus.xlsx')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
